# Как научить компьютер читать? 

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы. 

In [1]:
# Ссылка на выбранное вами произведение
# Я взял преступление и наказание (ненвижу Достоевского)
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'


Спарсим текст из файлика.

In [2]:
import requests

resp = requests.get(url)
text = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы. 

In [3]:
text = text.lower()

Разобьём весь текст на предложения. 

In [4]:
import re 
# выкидываем лишние символы! 
text = re.sub('\n|\t|\r', ' ', text)

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ElenTevanyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ElenTevanyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)
len(sents)

13702

In [8]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [9]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[220])

['действительно',
 'на',
 'его',
 'платье',
 'и',
 'даже',
 'в',
 'волосах',
 'кое',
 'где',
 'виднелись',
 'прилипшие',
 'былинки',
 'сена']

In [10]:
# разбейте все предложения на токены 
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]

In [11]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [12]:
len(words) # всего слов

173403

In [13]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова. 

In [14]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [15]:
len(stopwords_ru)

151

In [16]:
# избавьтесь от стоп-слов 
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                       for sent in sents_tokenize ]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [17]:
! pip install pymorphy2

     |████████████████████████████████| 55 kB 1.2 MB/s eta 0:00:011
     |████████████████████████████████| 8.2 MB 2.7 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=f3c7fd692edc17d44a525121aa3409de481ee4b408bd764a9506c6345cebb5da
  Stored in directory: /Users/ElenTevanyan/Library/Caches/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [18]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

text = "Элен и ребята не попали во Францию!"
tokens = tokenizer.tokenize(text)

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авеньон и пленить папа'

In [19]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

In [20]:
morph.normal_forms('стали')

['стать', 'сталь']

Обработаем все слова из датасета. 

In [21]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent] 
                    for sent in sents_tokenize]

In [22]:
# Flatten без numpy :) 
words = [item for sent in  sents_tokenize for item in sent]

In [23]:
len(words) # всего слов

93069

In [24]:
len(set(words)) # уникальных слов

11084

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочешь больше предобработки, [читай мой мануал](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb) об этом.  Давайте построим словарик с частотностями и перейдём к моделированию. 

In [25]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [26]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :) 

4768

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом 
* `size` — размер вектора, 
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `iter` — количество итераций,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

In [ ]:
!pip install gensim

In [30]:
#%%time 
from gensim.models.word2vec import Word2Vec

# size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели 
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке 
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много! 

(0, 0)

In [31]:
model.corpus_count # число примеров в обучающей выборке

13702

Смотрим, сколько в модели слов.

In [32]:
len(model.wv.vocab)

4768

In [33]:
'старуха' in model.wv.vocab

True

## 3. Свойства модели

In [34]:
# вектор слова
model.wv['старуха'][:10]

array([-1.2857313e-03,  1.5679699e-03,  1.1595329e-05,  1.2094073e-03,
       -2.9998769e-03,  1.5821587e-04, -4.7699031e-03,  1.0220370e-04,
        4.7137192e-03,  5.7172548e-04], dtype=float32)

In [35]:
# размерность вектора
model.wv['старуха'].shape

(100,)

In [36]:
# похожести слов 
model.wv.similarity('тварь', 'право')

-0.10447187

In [37]:
# самые похожие
model.wv.most_similar('топор')

[('служанка', 0.3315965533256531),
 ('прервать', 0.33039742708206177),
 ('отыскивать', 0.32917648553848267),
 ('курс', 0.3291257619857788),
 ('слишком', 0.3230099081993103),
 ('повести', 0.3115658760070801),
 ('язык', 0.3104204535484314),
 ('пять', 0.299183189868927),
 ('четырнадцать', 0.294633150100708),
 ('разделяться', 0.29008638858795166)]

In [38]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'], 
                       negative=['тварь'])[:10]

[('притворить', 0.35559073090553284),
 ('кхи', 0.34767094254493713),
 ('бледно', 0.3405333161354065),
 ('завлечь', 0.3353060483932495),
 ('щи', 0.3299151062965393),
 ('предчувствие', 0.32079312205314636),
 ('поносить', 0.31370052695274353),
 ('стакан', 0.31055742502212524),
 ('ограбить', 0.30707839131355286),
 ('изломанный', 0.3036160171031952)]

## 4. Как дообучить модель? 

Ради чистоты эксперимента сохраним текущую модель и заново подгрузим её. 

In [39]:
model_path = "./our_w2v.model"
model.save(model_path)

In [40]:
our_model = Word2Vec.load(model_path)

Подгрузим другое произведение и сделаем для него предобработку. 

In [66]:
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/vector-school/SkazkaOCareSaltane.txt'

resp = requests.get(url)
text2 = resp.text 

# Последние 500 символов. Аккуратно! Спойлеры!
print(text2[-500:])

 узнает...
В нем взыграло ретивое!
"Что я вижу? что такое?
Как!" - и дух в нем занялся...
Царь слезами залился,
Обнимает он царицу,
И сынка, и молодицу,



И садятся все за стол;
И веселый пир пошел.
А ткачиха с поварихой,
С сватьей бабой Бабарихой
Разбежались по углам;
Их нашли насилу там.
Тут во всем они признались,
Повинились, разрыдались;
Царь для радости такой
Отпустил всех трех домой.
День прошел - царя Салтана
Уложили спать вполпьяна.
Я там был; мед, пиво пил -
И усы лишь обмочил.

1831




Предобработка.

In [67]:
text2 = text2.lower()
sents2 = sent_tokenize(text2)

sents_tokenize2 = [tokenizer.tokenize(sent) for sent in sents2]
sents_tokenize2 = [[morph.normal_forms(word)[0] for word in text_cur if word not in stopwords_ru]
                      for text_cur in sents_tokenize2]

In [68]:
sents_tokenize2[10]

['сени', 'выйти', 'царь', 'отец']

In [69]:
len(sents_tokenize2)

254

Дополняем модель.

In [70]:
# обновили словарь
our_model.build_vocab(sents_tokenize2, update=True)

# дообучили модель
our_model.train(sents_tokenize2, 
                total_examples=our_model.corpus_count, 
                epochs=100)

(0, 0)

In [71]:
'ядро' in model.wv.vocab

False

In [72]:
'ядро' in our_model.wv.vocab

True

In [73]:
our_model.wv.most_similar('ядро')

[('проседь', 0.33910953998565674),
 ('содержимый', 0.33011293411254883),
 ('кусать', 0.32447952032089233),
 ('припомниться', 0.3141816556453705),
 ('радостный', 0.30989477038383484),
 ('пятнадцать', 0.309553325176239),
 ('хвататься', 0.3031322658061981),
 ('иерусалим', 0.3015998601913452),
 ('вес', 0.2926264703273773),
 ('сапог', 0.2899753451347351)]

Пример со старым словом.

In [74]:
our_model.wv.most_similar('сын')

[('внезапно', 0.3412628769874573),
 ('история', 0.3265072703361511),
 ('серьёзный', 0.322419673204422),
 ('стаканчик', 0.3128495216369629),
 ('новичок', 0.30967897176742554),
 ('четыре', 0.3076416850090027),
 ('неуклюжий', 0.3046972155570984),
 ('кирпич', 0.29859185218811035),
 ('поправиться', 0.2891137897968292),
 ('университетский', 0.28426387906074524)]

In [75]:
model.wv.most_similar('сын')

[('внезапно', 0.3412628769874573),
 ('история', 0.3265072703361511),
 ('серьёзный', 0.322419673204422),
 ('стаканчик', 0.3128495216369629),
 ('новичок', 0.30967897176742554),
 ('четыре', 0.3076416850090027),
 ('неуклюжий', 0.3046972155570984),
 ('кирпич', 0.29859185218811035),
 ('поправиться', 0.2891137897968292),
 ('университетский', 0.28426387906074524)]